In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.insert(0, '/mnt/data/molchanov/dltranz')

from domyshnik.models import *
from domyshnik.data import *
from domyshnik.constants import *
from domyshnik.utils import *


def draw(imgs):
        if isinstance(imgs, list):
            imgs = torch.stack(imgs)
        fig = plt.figure()
        rows, columns = 1, imgs.shape[0]
        for i in range(imgs.shape[0]):
            fig.add_subplot(rows, columns, i+1)
            plt.imshow(imgs[i])
        plt.show()
        
%matplotlib inline

# Metric Learning MNIST Test 

In [ ]:
#model = get_mnist_metriclearning_model()
model = get_mnist_metriclearning_persample_model()
model

In [ ]:
data_loader = get_mnist_test_loader(BATCH_SIZE, n_augments=N_AUGMENTS)
sample = next(iter(data_loader))
data = sample[0].view(-1, sample[0].size(-2), sample[0].size(-1))
draw(sample[0][0])

In [ ]:
embeds = model(sample[0])

In [ ]:
distances = torch.matmul(embeds, embeds.transpose(0, 1))
for i in range(distances.shape[0]):
    distances[i, i] = -100

In [ ]:
for i in range(distances.shape[0]):
    vals, idx = torch.sort(distances[i], descending=True)
    idx = [i] + idx[:N_AUGMENTS + 1].numpy().tolist()
    imgs = data[idx]
    draw(imgs)

# Domyshnik Mnist Test

In [ ]:
import torch.nn.functional as F

In [ ]:
data_loader = get_mnist_test_loader(BATCH_SIZE, n_augments=N_AUGMENTS, augment_labels=True)
sample = next(iter(data_loader))
data = sample[0].view(-1, sample[0].size(-2), sample[0].size(-1))
draw(sample[0][0])

In [ ]:
sample[1]

# experiment 1 (domyshnik net)

In [ ]:
info = mnist_domyshnik_lunch_info
sample = next(iter(info.test_loader))

In [ ]:
model = get_mnist_domyshnik_model()
model

In [ ]:
out = model(sample[0])
out = out.view(BATCH_SIZE, N_AUGMENTS + 1, out.size(-1))
out = F.softmax(out, dim=-1)
out.size()


In [ ]:
draw(sample[0][0])

In [ ]:
out0 = out[0]
dists = torch.matmul(out0, out0.transpose(0, 1))
'max', dists.max(), 'min', dists.min()

In [ ]:
draw(sample[0][10])

In [ ]:
out1 = out[1]
dists = torch.matmul(out0, out1.transpose(0, 1))
'max', dists.max(), 'min', dists.min()

In [ ]:
print(out[0, 0])
print(out[0, 1])
print(out[0, 2])
print(out[0, 3])

print(out[10, 0])
print(out[110, 1])
print(out[23, 2])
print(out[4, 3])

# experiment 2 metric learning simularity

In [ ]:
#m_model = get_mnist_metriclearning_model()
m_model = get_mnist_metriclearning_persample_model()
m_model

In [ ]:
m_out = m_model(sample[0])
m_out = m_out.view(BATCH_SIZE, N_AUGMENTS + 1, m_out.size(-1))
m_out.size()

### distance to negatives

In [ ]:
dists = F.pairwise_distance(m_out[0], m_out[10])
'max', dists.max().item(), 'min', dists.min().item()

### distance to self 

In [ ]:
dists = F.pairwise_distance(m_out[0], m_out[0][torch.randperm(m_out[0].size(0))])
'max', dists.max().item(), 'min', dists.min().item()
dists

In [ ]:
t = -1
for i in range(BATCH_SIZE):
    dists = F.pairwise_distance(m_out[i], m_out[i][torch.randperm(m_out0.size(0))])
    if t < dists.max().item():
        t = dists.max().item()
'max', t

In [ ]:
t = -1
for i in range(BATCH_SIZE):
    dists = F.pairwise_distance(m_out[0], m_out[i])
    if t < dists.max().item():
        t = dists.max().item()
'max', t

In [ ]:
m = MnistDomyshnikNetNet3()
m

In [ ]:
out = m(sample[0])
out.size()